In [4]:
import numpy as np

bbox_annos = np.load("./Google Drive/bbox_annos.npy", allow_pickle=True).item()
bbox_annos_sample = {key: bbox_annos[key] for key in list(bbox_annos.keys())[:10]}
bbox_annos_sample

{'fork_000000537944.jpg': [56, 229, 33, 49],
 'knife_000000233539.jpg': [372, 193, 114, 92],
 'tv_000000192651.jpg': [51, 138, 60, 54],
 'laptop_000000335308.jpg': [16, 210, 52, 39],
 'sink_000000164725.jpg': [158, 219, 159, 94],
 'fork_000000061672.jpg': [383, 101, 51, 141],
 'laptop_000000204979.jpg': [48, 98, 130, 101],
 'mouse_000000539056.jpg': [386, 206, 41, 44],
 'fork_000000323370.jpg': [399, 55, 65, 184],
 'tv_000000203160.jpg': [98, 57, 71, 71]}

In [ ]:
import numpy as np

bbox_annos = np.load("./Google Drive/bbox_annos.npy", allow_pickle=True).item()
bbox_annos_sample = {key: bbox_annos[key] for key in list(bbox_annos.keys())[:10]}
bbox_annos_sample